In [1]:
from dataset import ShapeNetDataModule2
import volumentations as vol


# data_module = ShapeNetDataModule(
data_module = ShapeNetDataModule2(
        "./data",
        target_categories='chair', #config.target_categories,
        batch_size=2, # vae_config.data.batch_size,
        num_workers=1, # vae_config.data.num_workers,
        max_num_images_per_cat=-1, # config.max_num_images_per_cat,
        transform=['flip'], # config.transform,
    )

In [2]:
dl = data_module.train_dataloader()
for batch in dl:
    break

In [3]:
batch[0].shape

torch.Size([2, 128, 128, 128])

In [19]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import torch
import h5py
from load_data import voxelize
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from io import BytesIO
from PIL import Image
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go

def get_voxel_plotly(voxel_grid, lim=128):
    """
    Visualizes a 3D binary voxel grid using Plotly.

    Parameters:
    voxel_grid (numpy.ndarray): A 3D binary voxel grid where 1 indicates occupancy and 0 indicates empty.
    """

    # Get the coordinates of occupied voxels
    occupied_voxels = np.argwhere(voxel_grid == 1)

    # Create a 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=occupied_voxels[:, 0],
        y=occupied_voxels[:, 2],
        z=occupied_voxels[:, 1],
        mode='markers',
        marker=dict(
            size=2,
            color='blue',
        )
    )])

    # Set labels
    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='data',
            xaxis=dict(range=[0, lim]),
            yaxis=dict(range=[0, lim]),
            zaxis=dict(range=[0, lim])
        )
    )

    return fig

# Use the new function to visualize the voxel grid
get_voxel_plotly(batch[0][0].numpy()).show()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
aug = vol.Compose([
    # vol.Flip(2, p=0.5),
    vol.Rotate((0, 0), (-15, 15), (0, 0), p=1),
])


out = (aug(image=batch[0][0].numpy())['image'] > 0.01).astype(np.float32)
print(out.shape)
get_voxel_plotly(out).show()


(128, 128, 128)


In [17]:
from utils import box_blur

get_voxel_plotly(batch[0][0].numpy()).show()
out = box_blur(batch[0][0].numpy())
get_voxel_plotly(out).show()
